## <center>NLP with manual classification</center> 

In [61]:
import pandas as pd
pd.set_option('display.max_colwidth', 250)
pd.set_option('max_rows', 2500)
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import resample

In [62]:
stopwords=['reformedesretraites','réformedesretraites','retraiter','grevegenerale','reformedesretraites']

In [63]:
greve_twitter_origin=pd.read_csv('greve_twitter_class.csv', index_col=[0])

In [64]:
greve_twitter_origin.tweet_processed2=[x.replace("""'""",'') for x in greve_twitter_origin.tweet_processed2]
greve_twitter_origin.date = greve_twitter_origin.date.map(lambda x: str(x)[:-15])

In [1]:
index_neg=[31,33,44,46,55,59,61,74,78,108,119,121,130,136,141,149,154,162,168,175,180,186,
           187,188,193,199,219,220,240,267,274,276,279,284,289,292,303,312,323,328,330,339,341,
           346,347,348,349,361,363,396,403,404,417,440,441,442,447,451,462,465,466,477,487,488,
           491,493,496,497,2,20,24,25,28,504,516,521,525,532,536,540,544,545,552,554,557,558,
           563,576,579,585,591,592,595,599,602,634,658,659,660,661,675,681,685,688,690,699,709,
          711,715,722,723,733,736,746,747,753,756,762,763,764,766,783,788,798,801,803,805,810,
           813,816,826,827,828,829,831,839,850,854,856,861,878,889,890,893,900,902,906,908,910,
           916,929,930,937,940,953,956,958,966,970,973,976,982,988,997,998,1001,1003,1006,1007,
           1008,1010,1011,1030,1031,1026,1034,1036,1037,1038,1039,1040,1042,1047,1049,1051,1053,
          1056,1057,1058,1060,1063,1064,1065,1067,1068,1069,1072,1074,1077,1081,1083,1084,1087,
          1091,1096,1097,1098,1100,1103,1108,1109,1116,117,1125,1145,1153,1154,1156,1158,1161,
          1169,1175,1178,1180,1192,1193,1203,1205,1207,1208,1210,1214,1237,1262,1264,1266,1279,
          1280,1282,1290,1293,1298,1305,1314,1316,1343,1345,1350,1351,1352,1353,1354,1356,1357,
           1361,1368,1374,1375,1377,1379,1380,1389,1396,1398,1400,1404,1407,1408,1410,1415,1421,
           1423,1424,1427,1430,1431,1433,1434,1437,1441,1442,1445,1453,1458,1468,1474,1485,1486,
           1493,1494,1495,1500,1501,1507,1509,1512,1517,1522,1527,1528,1530,1543,1544,1550,1553,
          1564,1565,1568,1569,1570,1574,1576,1583,1584,1597,1598,1600,1601,1606,1612,1617,1621,
          1623,1626,1626,1628,1630,1631,1634,1637,1645,1648,1650,1657,1664,1648,1650,1657,1664,
          1666,1669,1672,1673,1674,1682,1683,1690,1694,1696,1698,1700,1701,1702,1704,1705,1706,
          1707,1708,1710,1716,1717,1721,1732,1733,1736,1739,1748,1758,1776,1778,1782,1784,1785,
          1794, 1797,1799,1814,1826,1832,1835,1838,1839,1840,1842,1846,1849,1854,1856,1882,1883,
           1887,1901,1906,1907,1914,1917,1918,1923,1924,1928,1932,1935,1936,1942,1944,1947,1953,
          1954,1955,1960,1961,1967,1968,1972,1975,1983,1984,1987,1990,1992,1997,2001,2009,2012,
          2017,2020,2022,2033,2040,2041,2042,2045,2047,2048,2053,2056,2060,2062,2063,2064,2065,
          2070,2081,2084,2085,2088,2105,2106,2119,2128,2129,2130,2131,2134,2135,2139,2140,2143,
          2149,2158,2159,2163,2173,2175,2177,2178,2179,2181,2189,2190,2191,2192,2195,2200,2206,
          2207,2215,2221,2224,2226,2228,2233,2235,2240,2241,2242,2243,2249,2252,2253,2256,2257,
          2261,2263,2269,2275,2276,2281,2295,2299,2300,2302,2305,2311,2312,2313,2321,2322,2325,
          2326,2329,2330,2333,2336,2338,2340,2341,2342,2347,2349,2350,2352,2353,2354,2358,2360,
          2361,2364,2366,2367,2372,2379,2381,2383,2385,2394,2400,2402,2403,2405,2406,2411,2413,
          2417,2425,2431,2437,2440,2441,2448,2452,2453,2457,2461,2473,2485,2487,2488,2494,2497,
          2499,2503,2505,2507,2508,2511,2512,2518,2522,2524,2529,2535,2536,2543,2544,2545,2546,
          2547,2549,2551,2553,2554,2558,2561,2565,2569,2571,2572,2573,2574,2576,2583,2585,2586,
          2587,2588,2593,2600,2601,2607,2611,2612,2617,2625,2626,2629,2634,2636,2640,2641,2646,
          2651,2654,2655,2662,2663,2666,2667,2670,2675,2681,2684,2688,2691,2693,2695,2697,2703,
          2715,2718,2724,2729,2730,2731,2734,2746,2753,2754,2757,2764,2765,2766,2768,2776,2777,
          2778,2779,2781,2782,2792,2794,2797,2801,2802,2804,2806,2808,2811,2816,2817,2818,2824,
          2828,2829,2830,2832,2838,2839,2840,2842,2843,2851,2853,2856,2857,2858,2864,2872,2878,
           2894,2898,2899,2901,2902,2905,2909,2913,2919,2920,2924,2929,2931,2932,2936,2937,2939,
           2940,2941,2942,2945,2966,2968,2970,2975,2978,2986,2991,2996,2998]
index_pos=[14,22,56,79,122,163,195,237,242,253,400,407,408,410,434,435,439,443,457,459,479,
          528,569,617,666,741,755,855,919,942,974,975,1027,1055,1167,1179,1465,1513,1520,1614,
          1817,1818,1819,1820,2004,2021,2069,2137,2334,2610,2664,2696,2705,2739,2770,2796,2813]
greve_twitter_origin['polarity'].iloc[index_neg] = 'negatif'
greve_twitter_origin['polarity'].iloc[index_pos] = 'positif'


NameError: name 'greve_twitter_origin' is not defined

In [152]:
greve_twitter_train = greve_twitter_origin[0:2000]
greve_twitter_train['polarity'].value_counts()

negatif    1383
positif     617
Name: polarity, dtype: int64

In [154]:
df_majority = greve_twitter_train[greve_twitter_train.polarity=='negatif']
df_minority = greve_twitter_train[greve_twitter_train.polarity=='positif']
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=1383,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
greve_twitter_train = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
greve_twitter_train.polarity.value_counts()

negatif    1383
positif    1383
Name: polarity, dtype: int64

In [155]:
count_vectorizer = CountVectorizer(analyzer='word') 
cv = count_vectorizer.fit(greve_twitter_train['tweet_processed2'])
pickle.dump(cv, open("cv1.pkl", "wb"))
cv_ = count_vectorizer.fit_transform(greve_twitter_train['tweet_processed2'])
cv1 = pickle.load(open("cv1.pkl", 'rb'))
cv1_new = CountVectorizer(vocabulary = cv1.vocabulary_)

In [156]:
X_train_logreg,X_test_logreg,y_train_logreg,y_test_logreg = train_test_split(cv_,
                                                greve_twitter_train['polarity'] , test_size=.2,
                                                 random_state=42)

In [157]:
logreg = LogisticRegression(C=1,penalty="l2")
logreg.fit(X_train_logreg,y_train_logreg)
prediction_logreg = logreg.predict(X_test_logreg)
print(classification_report(prediction_logreg,y_test_logreg))

              precision    recall  f1-score   support

     negatif       0.80      0.94      0.86       243
     positif       0.95      0.81      0.88       311

   micro avg       0.87      0.87      0.87       554
   macro avg       0.87      0.88      0.87       554
weighted avg       0.88      0.87      0.87       554



In [2]:
X_train_xg,X_test_xg,y_train_xg,y_test_xg = train_test_split(cv_,greve_twitter_train['polarity'],
                                                             test_size=.2,random_state=42)
xgbc = XGBClassifier(max_depth=9, min_child_weight=4, n_estimators=1000, nthread= 3)
xgbc.fit(X_train_xg,y_train_xg)
prediction_xgb = xgbc.predict(X_test_xg)
print(classification_report(prediction_xgb,y_test_xg))

NameError: name 'train_test_split' is not defined

In [158]:
greve_twitter_test = greve_twitter_origin[3000:-1]
X_cv2 = cv1_new.transform(greve_twitter_test['tweet_processed2'])
greve_twitter_test['polarity']=logreg.predict(X_cv2)
greve_twitter = pd.concat([greve_twitter_train, greve_twitter_test], ignore_index=True)

In [ ]:
#grevetwitter = greve_twitter.drop(columns='polarity2',inplace=True)
greve_twitter.to_csv('greve_twitter_classif2.csv')